# Milestone 2.

## Installing dependencies

In [1]:
%pip install torch dgl networkx pandas numpy

     -------------------------------------- 167.3/167.3 MB 3.0 MB/s eta 0:00:00
     ---------------------------------------- 2.4/2.4 MB 6.4 MB/s eta 0:00:00
     ---------------------------------------- 2.0/2.0 MB 6.1 MB/s eta 0:00:00
     ---------------------------------------- 10.4/10.4 MB 4.1 MB/s eta 0:00:00
     ---------------------------------------- 14.6/14.6 MB 5.9 MB/s eta 0:00:00
     ---------------------------------------- 62.8/62.8 kB 3.5 MB/s eta 0:00:00
     -------------------------------------- 40.1/40.1 MB 502.0 kB/s eta 0:00:00
     ---------------------------------------- 78.5/78.5 kB 1.5 MB/s eta 0:00:00
     -------------------------------------- 498.1/498.1 kB 2.8 MB/s eta 0:00:00
     ------------------------------------ 140.6/140.6 kB 225.5 kB/s eta 0:00:00
     ---------------------------------------- 61.5/61.5 kB 1.7 MB/s eta 0:00:00
     -------------------------------------- 155.3/155.3 kB 4.7 MB/s eta 0:00:00
Note: you may need to restart the kernel to 

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [10]:
pip install -U scikit-learn


     ---------------------------------------- 8.2/8.2 MB 415.7 kB/s eta 0:00:00
     ------------------------------------ 298.0/298.0 kB 708.7 kB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


## Setting hyperparameters

In [26]:
epochs = 100
features_num = 10
hidden1 = 64
hidden2 = 32
learning_rate = 0.05

In [27]:
# Set torch device
import torch

device = torch.device("cpu") # training on CPU because of runtime errors

## Defining the model according to the code from [`dmlc/dgl`](https://github.com/dmlc/dgl)

The code below will be taken from the reference implementation in the Deep Learning Graph Library (DGL). It follows the architecture shown in the source article.

In [42]:
import torch.nn as nn
import torch.nn.functional as F
from dgl.nn.pytorch import GraphConv # must be installed with pip

class VGAEModel(nn.Module):
    def __init__(self, in_dim, hidden1_dim, hidden2_dim):
        super(VGAEModel, self).__init__()
        self.in_dim = in_dim
        self.hidden1_dim = hidden1_dim
        self.hidden2_dim = hidden2_dim

        layers = [
            GraphConv(
                self.in_dim,
                self.hidden1_dim,
                activation=F.relu,
                allow_zero_in_degree=True,
            ),
            GraphConv(
                self.hidden1_dim,
                self.hidden2_dim,
                activation=lambda x: x,
                allow_zero_in_degree=True,
            ),
            GraphConv(
                self.hidden1_dim,
                self.hidden2_dim,
                activation=lambda x: x,
                allow_zero_in_degree=True,
            ),
            GraphConv(
                32,
                16,
                activation=lambda x: x,
                allow_zero_in_degree=True,
            ),
        ]
        self.layers = nn.ModuleList(layers)

    def encoder(self, g, features):
        h = self.layers[0](g, features)
        self.mean = self.layers[1](g, h)
        self.log_std = self.layers[2](g, h)
        gaussian_noise = torch.randn(features.size(0), self.hidden2_dim).to(
            device
        )
        sampled_z = self.mean + gaussian_noise * torch.exp(self.log_std).to(
            device
        )
        return sampled_z

    def decoder(self, z):
        adj_rec = torch.sigmoid(torch.matmul(z, z.t()))
        return adj_rec

    def forward(self, g, features):
        z = self.encoder(g, features)
        adj_rec = self.decoder(z)
        return adj_rec

## Helper functions for data preprocessing 

In [43]:
import scipy.sparse as sp
import numpy as np

def sparse_to_tuple(sparse_mx):
    if not sp.isspmatrix_coo(sparse_mx):
        sparse_mx = sparse_mx.tocoo()
    coords = np.vstack((sparse_mx.row, sparse_mx.col)).transpose()
    values = sparse_mx.data
    shape = sparse_mx.shape
    return coords, values, shape

def mask_test_edges(adj):
    # Remove diagonal elements
    adj = adj - sp.dia_matrix(
        (adj.diagonal()[np.newaxis, :], [0]), shape=adj.shape
    )
    adj.eliminate_zeros()
    # Check that diag is zero:
    assert np.diag(adj.todense()).sum() == 0

    adj_triu = sp.triu(adj)
    adj_tuple = sparse_to_tuple(adj_triu)
    edges = adj_tuple[0]
    edges_all = sparse_to_tuple(adj)[0]
    num_test = int(np.floor(edges.shape[0] / 10.0))
    num_val = int(np.floor(edges.shape[0] / 20.0))

    all_edge_idx = list(range(edges.shape[0]))
    np.random.shuffle(all_edge_idx)
    val_edge_idx = all_edge_idx[:num_val]
    test_edge_idx = all_edge_idx[num_val : (num_val + num_test)]
    test_edges = edges[test_edge_idx]
    val_edges = edges[val_edge_idx]
    train_edges = np.delete(
        edges, np.hstack([test_edge_idx, val_edge_idx]), axis=0
    )

    def ismember(a, b, tol=5):
        rows_close = np.all(np.round(a - b[:, None], tol) == 0, axis=-1)
        return np.any(rows_close)

    test_edges_false = []
    while len(test_edges_false) < len(test_edges):
        idx_i = np.random.randint(0, adj.shape[0])
        idx_j = np.random.randint(0, adj.shape[0])
        if idx_i == idx_j:
            continue
        if ismember([idx_i, idx_j], edges_all):
            continue
        if test_edges_false:
            if ismember([idx_j, idx_i], np.array(test_edges_false)):
                continue
            if ismember([idx_i, idx_j], np.array(test_edges_false)):
                continue
        test_edges_false.append([idx_i, idx_j])

    val_edges_false = []
    while len(val_edges_false) < len(val_edges):
        idx_i = np.random.randint(0, adj.shape[0])
        idx_j = np.random.randint(0, adj.shape[0])
        if idx_i == idx_j:
            continue
        if ismember([idx_i, idx_j], train_edges):
            continue
        if ismember([idx_j, idx_i], train_edges):
            continue
        if ismember([idx_i, idx_j], val_edges):
            continue
        if ismember([idx_j, idx_i], val_edges):
            continue
        if val_edges_false:
            if ismember([idx_j, idx_i], np.array(val_edges_false)):
                continue
            if ismember([idx_i, idx_j], np.array(val_edges_false)):
                continue
        val_edges_false.append([idx_i, idx_j])

    data = np.ones(train_edges.shape[0])

    # Re-build adj matrix
    adj_train = sp.csr_matrix(
        (data, (train_edges[:, 0], train_edges[:, 1])), shape=adj.shape
    )
    adj_train = adj_train + adj_train.T

    return (
        adj_train,
        train_edges,
        val_edges,
        val_edges_false,
        test_edges,
        test_edges_false,
    )

def preprocess_graph(adj):
    adj = sp.coo_matrix(adj)
    adj_ = adj + sp.eye(adj.shape[0])
    rowsum = np.array(adj_.sum(1))
    degree_mat_inv_sqrt = sp.diags(np.power(rowsum, -0.5).flatten())
    adj_normalized = (
        adj_.dot(degree_mat_inv_sqrt)
        .transpose()
        .dot(degree_mat_inv_sqrt)
        .tocoo()
    )
    return adj_normalized, sparse_to_tuple(adj_normalized)

## Loading our data

The `load_data` function uses our own data as reference. It expects the `data/` folder to have a `features.csv` (see Milestone I.)

In [44]:
import numpy as np
import pickle as pkl
import networkx as nx
import scipy.sparse as sp
import pandas as pd


def load_data():

    data = pd.read_csv("data/features.csv", header = None)
    data[0] = data[0].str.lstrip("D, B, 0")
    data[0] = data[0].astype('int')
    adj2 = data[0].copy()
    data = data.set_index(data.columns[0])
    sp.csr_matrix(data.values)

    graph2 = pd.read_csv("db/ChCh-Miner_durgbank-chem-chem.tsv.gz", header = None)
    graph = graph2.copy()
    graph[0] = graph2[0].str.split('\t').str[0]
    graph[1] = graph2[0].str.split('\t').str[1]
    graph[0] = graph[0].str.lstrip("D, B, 0")
    graph[0] = graph[0].astype('int')
    graph[1] = graph[1].str.lstrip("D, B, 0")
    graph[1] = graph[1].astype('int')
    graph

    adj = pd.DataFrame(adj2)
    adj = adj.set_index(adj.columns[0])
    adj[0] = np.empty((len(adj), 0)).tolist()

    for g in graph.iterrows():
        if g[1][0] in adj.index:
            if g[1][1] in adj.index:
                adj[0].loc[g[1][0]].insert(g[1][0],g[1][1])
                adj[0].loc[g[1][1]].insert(g[1][1],g[1][0])

    for a in adj.iterrows():
        if a[1][0] == []:
            adj = adj.drop(index = a[0])
            data = data.drop(index = a[0])

    features = sp.csr_matrix(data.values)/100.0
    adj_dict = pd.Series(adj[0], index = adj.index).to_dict()
    adjacency = nx.adjacency_matrix(nx.from_dict_of_lists(adj_dict))

    return adjacency, features

Preprocessing steps mainly taken from the repository.

In [45]:
import dgl

adj, features = load_data()
features = sparse_to_tuple(features.tocoo())

# Store original adjacency matrix (without diagonal entries) for later
adj_orig = adj
adj_orig = adj_orig - sp.dia_matrix(
    (
        adj_orig.diagonal()[np.newaxis, :], [0]
    ), shape=adj_orig.shape
)
adj_orig.eliminate_zeros()

(
    adj_train,
    train_edges,
    val_edges,
    val_edges_false,
    test_edges,
    test_edges_false,
) = mask_test_edges(adj)
adj = adj_train

# Some preprocessing
adj_normalization, adj_norm = preprocess_graph(adj)

    # Create model
graph = dgl.from_scipy(adj_normalization)
graph.add_self_loop()

    # Create Model
pos_weight = float(adj.shape[0] * adj.shape[0] - adj.sum()) / adj.sum()
norm = (
    adj.shape[0]
    * adj.shape[0]
    / float((adj.shape[0] * adj.shape[0] - adj.sum()) * 2)
)

adj_label = adj_train + sp.eye(adj_train.shape[0])
adj_label = sparse_to_tuple(adj_label)

adj_norm = torch.sparse.FloatTensor(
    torch.LongTensor(adj_norm[0].T),
    torch.FloatTensor(adj_norm[1]),
    torch.Size(adj_norm[2]),
)
adj_label = torch.sparse.FloatTensor(
    torch.LongTensor(adj_label[0].T),
    torch.FloatTensor(adj_label[1]),
    torch.Size(adj_label[2]),
)
features = torch.sparse.FloatTensor(
    torch.LongTensor(features[0].T),
    torch.FloatTensor(features[1]),
    torch.Size(features[2]),
)

weight_mask = adj_label.to_dense().view(-1) == 1
weight_tensor = torch.ones(weight_mask.size(0))
weight_tensor[weight_mask] = pos_weight

features = features.to_dense()
features_num = features.shape[-1]

def get_scores(edges_pos, edges_neg, adj_rec):
        def sigmoid(x):
            return 1 / (1 + np.exp(-x))

        # Predict on test set of edges
        preds = []
        pos = []
        for e in edges_pos:
            # print(e)
            # print(adj_rec[e[0], e[1]])
            preds.append(sigmoid(adj_rec[e[0], e[1]].item()))
            pos.append(adj_orig[e[0], e[1]])

        preds_neg = []
        neg = []
        for e in edges_neg:
            preds_neg.append(sigmoid(adj_rec[e[0], e[1]].data))
            neg.append(adj_orig[e[0], e[1]])

        preds_all = np.hstack([preds, preds_neg])
        labels_all = np.hstack([np.ones(len(preds)), np.zeros(len(preds_neg))])
        roc_score = roc_auc_score(labels_all, preds_all)
        ap_score = average_precision_score(labels_all, preds_all)

        return roc_score, ap_score


def get_acc(adj_rec, adj_label):
    labels_all = adj_label.to_dense().view(-1).long()
    preds_all = (adj_rec > 0.5).view(-1).long()
    accuracy = (preds_all == labels_all).sum().float() / labels_all.size(0)
    return accuracy

C:\Users\bence\AppData\Local\Temp\ipykernel_10364\964416747.py:44: FutureWarning: adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
  adjacency = nx.adjacency_matrix(nx.from_dict_of_lists(adj_dict))


## Creating the model and the optimizer

In [46]:
vgae_model = VGAEModel(features_num, hidden1, hidden2)
# create training component
optimizer = torch.optim.Adam(vgae_model.parameters(), lr=learning_rate)

In [47]:
from sklearn.metrics import average_precision_score, roc_auc_score

for epoch in range(epochs):

    # Training and validation using a full graph
    vgae_model.train()
    logits = vgae_model.forward(graph, features)

    # compute loss
    loss = norm * F.binary_cross_entropy(
        logits.view(-1), adj_label.to_dense().view(-1), weight=weight_tensor
    )
    kl_divergence = (
        0.5
        / logits.size(0)
        * (
            1
            + 2 * vgae_model.log_std
            - vgae_model.mean**2
            - torch.exp(vgae_model.log_std) ** 2
        )
        .sum(1)
        .mean()
    )
    loss -= kl_divergence

    # backward
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    train_acc = get_acc(logits, adj_label)
    val_roc, val_ap = get_scores(val_edges, val_edges_false, logits)
    print(f'Epoch: {epoch+1}\ttrain_loss={loss.item():.5f}, train_acc={train_acc:.5f} val_roc={val_roc:.5f} val_ap={val_ap:.5f}')

test_roc, test_ap = get_scores(test_edges, test_edges_false, logits)
print(
    f'End of training!\ntest_roc={test_roc:.5f}\ntest_ap={test_ap:.5f}'
)

Epoch: 1	train_loss=1199492255575965763108864.00000, train_acc=0.31998 val_roc=0.53340 val_ap=0.58838
Epoch: 2	train_loss=297224838663756578816.00000, train_acc=0.06918 val_roc=0.41602 val_ap=0.61604
Epoch: 3	train_loss=7006092490235183104.00000, train_acc=0.15584 val_roc=0.67828 val_ap=0.69654
Epoch: 4	train_loss=11653567479808.00000, train_acc=0.17141 val_roc=0.71765 val_ap=0.72049
Epoch: 5	train_loss=31227.90625, train_acc=0.11562 val_roc=0.79562 val_ap=0.78409
Epoch: 6	train_loss=6.13615, train_acc=0.13577 val_roc=0.82421 val_ap=0.79757
Epoch: 7	train_loss=186953.62500, train_acc=0.14591 val_roc=0.83062 val_ap=0.79561
Epoch: 8	train_loss=5.87275, train_acc=0.13981 val_roc=0.83317 val_ap=0.80820
Epoch: 9	train_loss=4.74576, train_acc=0.14507 val_roc=0.83206 val_ap=0.79558
Epoch: 10	train_loss=4.26832, train_acc=0.13891 val_roc=0.82996 val_ap=0.77946
Epoch: 11	train_loss=3.79865, train_acc=0.17372 val_roc=0.82438 val_ap=0.77920
Epoch: 12	train_loss=3.94152, train_acc=0.20599 val_roc=

In [48]:
print("Final area under the ROC curve (AUC):" , val_roc)
print("Final avarage precision (AP):", val_ap)

Final area under the ROC curve (AUC): 0.8375330884095715
Final avarage precision (AP): 0.8250215729704059


In this subject area the most common evaluations are the AUC (area under the ROC curve) and AP (avarage precision).
The AP on the first epoch was close to 50 %, meaning it was almost random, but for the 10th epoch it became 60% (which isn`t too much, but it will be better).

For more evaluation we choose the AUC metric. The reason behind it is, that the most important part of the project is to find all the interactions between drugs correctly if possible. So this metric checks that how precise is an interaction predicted by us. Meaning, if we predicted that between two drugs there is an interaction, what is the chance that there really is one. We chose this because it focuses on the finding more edges correctly, rather than finding non-edges correctly. 
As seen above this metric also went above 50% after a few epochs so it is working correctly. 